In [1]:
import pandas as pd
import csv
from surprise import Reader, Dataset, SVD, evaluate
import warnings; warnings.simplefilter('ignore')

In [2]:
data = pd.read_csv("dataset/movies_metadata.csv")
credits = pd.read_csv("dataset/credits.csv")
keywords = pd.read_csv("dataset/keywords.csv")
data = data.merge(credits, on='id')
data = data.merge(keywords, on='id')
data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
4,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...","[{'id': 642, 'name': 'robbery'}, {'id': 703, '..."


In [3]:
def get_genres(x):
    genre = []
    for i in x:
        genre.append(i['name'])
    return genre

In [4]:
def get_cast(x):
    cast = []
    num = 0
    for i in x:
        i['name'] = i['name'].replace(" ","")
        cast.append(i['name'])
        num = num + 1
        if num==5:
            break
    return cast

In [5]:
def get_crew(x):
    crew = []
    for i in x:
        if i['job'] == 'Director' or i['job'] =='Screenplay':
            i['name'] = i['name'].replace(" ","")
            crew.append(i['name'])
    return crew

In [6]:
def get_keywords(x):
    keywords = []
    for i in x:
        i['name'] = i['name'].replace(" ","")
        keywords.append(i['name'])
    return keywords

In [7]:
from ast import literal_eval

data['genres'] = data['genres'].apply(literal_eval).apply(get_genres)
data['cast'] = data['cast'].apply(literal_eval).apply(get_cast)
data['crew'] = data['crew'].apply(literal_eval).apply(get_crew)
data['keywords'] = data['keywords'].apply(literal_eval).apply(get_keywords)

data['soup'] = data['genres']+data['cast']+data['crew']+data['keywords']
data['soup'] = data['soup'].apply(lambda x: ' '.join(x))

data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords,soup
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,"[TomHanks, TimAllen, DonRickles, JimVarney, Wa...","[JohnLasseter, JossWhedon, AndrewStanton, Joel...","[jealousy, toy, boy, friendship, friends, riva...",Animation Comedy Family TomHanks TimAllen DonR...
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[RobinWilliams, JonathanHyde, KirstenDunst, Br...","[JonathanHensleigh, JoeJohnston, GregTaylor, J...","[boardgame, disappearance, basedonchildren'sbo...",Adventure Fantasy Family RobinWilliams Jonatha...
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[WalterMatthau, JackLemmon, Ann-Margret, Sophi...",[HowardDeutch],"[fishing, bestfriend, duringcreditsstinger, ol...",Romance Comedy WalterMatthau JackLemmon Ann-Ma...
3,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[SteveMartin, DianeKeaton, MartinShort, Kimber...","[NancyMeyers, AlbertHackett, CharlesShyer]","[baby, midlifecrisis, confidence, aging, daugh...",Comedy SteveMartin DianeKeaton MartinShort Kim...
4,False,NaN,60000000,"[Action, Crime, Drama, Thriller]",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,"[AlPacino, RobertDeNiro, ValKilmer, JonVoight,...","[MichaelMann, MichaelMann]","[robbery, detective, bank, obsession, chase, s...",Action Crime Drama Thriller AlPacino RobertDeN...


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
count = CountVectorizer(analyzer='word', stop_words='english')
matrix = count.fit_transform(data['soup'])
similarity = cosine_similarity(matrix,matrix)

In [9]:
ratings = pd.read_csv("dataset/ratings.csv")
ratings.head()

,userId,movieId,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1246,5.0
4,1,1968,4.0


In [10]:
reader = Reader()
dataset = Dataset.load_from_df(ratings, reader)
dataset.split(n_folds=5)
svd = SVD()
evaluate(svd, dataset, measures=['RMSE','MAE'])

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.8976
MAE:  0.6960
------------
Fold 2
RMSE: 0.8856
MAE:  0.6875
------------
Fold 3
RMSE: 0.9578
MAE:  0.7285
------------
Fold 4
RMSE: 0.9266
MAE:  0.7350
------------
Fold 5
RMSE: 0.9259
MAE:  0.7217
------------
------------
Mean RMSE: 0.9187
Mean MAE : 0.7137
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.8976474254214283,
                             0.885604110928975,
                             0.95782622303919,
                             0.9265520079030285,
                             0.9258964688969684],
                            'mae': [0.6960125465057829,
                             0.6874902223181822,
                             0.7285329337445917,
                             0.7349877436250132,
                             0.721719671449044]})

In [11]:
trainset = dataset.build_full_trainset()
svd.train(trainset)

In [12]:
titles = data[['title','vote_count','vote_average','id']]
index = pd.Series(data.index, index = data['id'])

In [13]:
def get_movie_list(movieId):
    ind = index[movieId]
    score = list(enumerate(similarity[ind]))
    score = sorted(score, key=lambda x: x[1], reverse=True)
    score = score[0:11]
    movie_indices = [i[0] for i in score]
    return movie_indices

In [14]:
def get_recommendation(userId):
    userdata = ratings[ratings['userId'] == userId]
    userdata = userdata[userdata['rating'] >= 2.5]
    if userdata.empty == True:
        return """Insufficient Data Present"""
    
    movie = set()
    for i in userdata['movieId']:
        movies = get_movie_list(i)
        for x in movies:
            movie.add(x)
    for i in userdata['movieId']:
        movie.remove(index[i])
        
    movie_indices = list(movie)
    top = pd.DataFrame(titles.iloc[movie_indices])
    top['rating'] = top['id'].apply(lambda x: svd.predict(userId, x).est)
    top = top.sort_values('rating', ascending=False)
    return top.head(10)

In [15]:
def add_rating(userId, title, rating):
    if rating>5 or rating<=0:
        return """rating should be between 0-5"""
    try:
        ind = data[data['title'] == title].id.item()
        row = [userId, ind, rating]
        ratings.loc[len(ratings)] = row
        row = [userId, ind, rating]
        with open('dataset/ratings.csv','a') as f:
            writer = csv.writer(f)
            writer.writerow(row)
    except:
        return """No movie found with this name"""

In [16]:
get_recommendation(1)

,title,vote_count,vote_average,id,rating
1219,Rocky IV,984.0,6.6,1374,4.111472
727,Michael,174.0,5.5,2928,3.685316
928,Rocky,1843.0,7.5,1366,3.600299
2113,Driven,180.0,4.5,10477,3.586076
4704,27 Dresses,1013.0,6.1,6557,3.586076
7114,Tangled Ever After,383.0,7.1,82881,3.586076
1867,Boys and Girls,107.0,5.4,10571,3.586076
1228,You've Got Mail,859.0,6.3,9489,3.586076
4176,Leprechaun 2,51.0,4.8,18009,3.586076
859,The Wedding Singer,603.0,6.5,11003,3.586076


In [17]:
get_recommendation(2)

,title,vote_count,vote_average,id,rating
12,Cutthroat Island,137.0,5.7,1408,3.723552
3978,Broken Flowers,359.0,6.8,308,3.662276
1976,Red Planet,271.0,5.4,8870,3.638267
1558,Women on the Verge of a Nervous Breakdown,149.0,7.6,4203,3.515768
3543,Bad Education,223.0,7.1,140,3.506791
1134,Edward Scissorhands,3731.0,7.5,162,3.479309
573,Brazil,877.0,7.6,68,3.436108
876,Live Flesh,89.0,7.2,267,3.425128
1308,The Matrix,9079.0,7.9,603,3.423405
654,Heathers,387.0,7.3,2640,3.423113


In [18]:
get_recommendation(3)

,title,vote_count,vote_average,id,rating
3579,Los Olvidados,70.0,7.9,800,3.838673
3978,Broken Flowers,359.0,6.8,308,3.602434
727,Michael,174.0,5.5,2928,3.374530
5121,Management,102.0,5.5,21583,3.294204
5699,Welcome to the Rileys,141.0,6.5,31007,3.294204
6358,Wake in Fright,66.0,7.6,26405,3.294204
6992,The Man from the Future,69.0,6.7,73624,3.294204
1228,You've Got Mail,859.0,6.3,9489,3.294204
1730,Heart and Souls,84.0,6.6,12187,3.294204
1474,Body Heat,128.0,7.1,14412,3.294204


In [19]:
get_recommendation(4)

,title,vote_count,vote_average,id,rating
4378,The Good Shepherd,342.0,6.3,1247,4.250547
2524,Changing Lanes,306.0,5.9,1537,4.198251
1134,Edward Scissorhands,3731.0,7.5,162,4.142740
1302,Forces of Nature,141.0,5.1,1641,4.085622
3945,Batman Begins,7511.0,7.5,272,4.067408
709,Batman Returns,1706.0,6.6,364,4.002808
4389,Letters from Iwo Jima,553.0,7.2,1251,3.982674
4872,The Mummy: Tomb of the Dragon Emperor,1418.0,5.2,1735,3.861230
7979,Batman v Superman: Dawn of Justice,7189.0,5.7,209112,3.825813
319,Batman,2145.0,7.0,268,3.823008


In [20]:
get_recommendation(5)    

,title,vote_count,vote_average,id,rating
2527,Human Nature,74.0,6.1,441,4.302787
610,The Terminator,4208.0,7.4,218,4.017728
727,Michael,174.0,5.5,2928,3.938296
4467,Reign Over Me,340.0,7.1,2355,3.823956
1782,The Virgin Suicides,841.0,7.1,1443,3.749848
8956,Trespass Against Us,59.0,6.0,256569,3.749848
2270,Next of Kin,53.0,5.0,27418,3.749848
2304,The Shootist,85.0,7.3,12584,3.749848
1817,American Gigolo,137.0,5.9,2768,3.749848
3950,Bewitched,463.0,4.8,9722,3.749848


In [21]:
get_recommendation(251)

,title,vote_count,vote_average,id,rating
3945,Batman Begins,7511.0,7.5,272,4.024592
5107,X-Men Origins: Wolverine,4086.0,6.2,2080,3.778002
770,Batman & Robin,1447.0,4.2,415,3.777005
709,Batman Returns,1706.0,6.6,364,3.763244
257,Killing Zoe,111.0,6.1,507,3.753720
7009,Captain America: The Winter Soldier,5881.0,7.6,100402,3.716179
4959,Farscape: The Peacekeeper Wars,74.0,7.5,13905,3.716179
4435,Reno 911!: Miami,72.0,5.6,10090,3.716179
5454,Iron Man 2,6969.0,6.6,10138,3.716179
1853,Soapdish,61.0,6.0,25562,3.716179


In [22]:
ratings[ratings['userId']==251]

,userId,movieId,rating
6216,251,49026,5.0
6217,251,155,5.0
6218,251,562,5.0
6219,251,13,3.5
6220,251,27205,4.5
6221,251,313369,1.5
6222,251,209112,5.0
6223,251,280690,1.0
6224,251,138832,4.0
6225,251,1726,4.0
